In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass, field

from typing import Dict, List, Tuple, Optional, Callable, Any

import math
import numpy as np
from PIL import Image

from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import BoundingBox2DLabel, BoundingBox2DAnnotation
from pysolotools.core.models import BoundingBox3DLabel, BoundingBox3DAnnotation
from pysolotools.core.models import Frame, Capture
from scipy.spatial.transform import Rotation as R

from torchvision.datasets import ImageFolder
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
from torchvision.models import swin_v2_b, Swin_V2_B_Weights
# from torch.utils.data import ConcatDataset, DataLoader
from collections import OrderedDict
from torch.utils.data import DataLoader

import torch
from torch import nn, Tensor

from torch.nn import functional as F

from torchvision.ops import FeaturePyramidNetwork, MLP, sigmoid_focal_loss
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor

import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.io import savemat

import reader
import utils
import network
import transformer
import cvmpca_2 as cvmpca
from torchvision.ops import FeaturePyramidNetwork
from my_trainer import SetCriterion
torch.set_float32_matmul_precision('medium')
# %matplotlib ipympl

c:\Users\LeonardNgo\miniconda3\envs\cvmpa\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
train_folder = 'D:/Unity/dataset/solo'
# training_dir = './data/train'
# testing_dir = './data/test'

In [4]:
train_loader = reader.UnityDataset.from_unity_to_loader(root=train_folder, batch_size=2)

# for sample in train_loader.dataset:
#     pass

for batch in train_loader:
    image_dicts, object_list = batch

    # for key, image in image_dicts.items():
    #     print(key, image.shape)
    # print('-'*30)
    # for targets in object_list:
    # print(object_list.position.shape)

    break

Dataset loaded succesfully.


In [ ]:
for batch in train_loader:
    image_dicts, object_list = batch

    # for key, image in image_dicts.items():
    #     print(key, image.shape)
    # print('-'*30)
    # for targets in object_list:
    for objs in object_list:
        print(objs.category.shape)

    break

In [ ]:
captures = train_loader.dataset.captures
cameras = {capture.id: utils.Camera.from_unity(capture) for capture in captures}
for key, item in image_dicts.items():
    print(key, item.shape)

In [ ]:
swin = transformer.Swin(is_trainable=True)
fpn = FeaturePyramidNetwork(swin.embed_dims, swin.embed_dim)
embed_dim = swin.embed_dim
embed_dims = swin.embed_dims

In [ ]:
keys = list(image_dicts.keys())  # camera ids
B = image_dicts[keys[0]].size(0)

visual_features = {
    camera_key: fpn({
        f'feat{i}': x.permute(0, 3, 1, 2)
        for i, x in enumerate(swin(images))
    })
    for camera_key, images in image_dicts.items()
}

In [ ]:
for key, items in visual_features.items():
    for key_, item in items.items():
        print(key, key_, item.shape)

In [ ]:
def create_grid(h, w):
    x = torch.arange(0, w, dtype=torch.float)
    y = torch.arange(0, h, dtype=torch.float)
    grid_x, grid_y = torch.meshgrid(x, y, indexing='ij')

    return torch.stack([grid_x.flatten(), grid_y.flatten(), torch.ones(h*w)], dim=-1)

cnn_layer = nn.Conv2d(3, 96, kernel_size=1, stride=1, padding=0)


for key, items in visual_features.items():
    print(key)
    for key_, item in items.items():
        B, D, H, W = item.shape
        grid = create_grid(H, W)#.unsqueeze(0).repeat(B, 1, 1, 1)
        print(grid.shape)
        rays = cameras[key].pix2ray(grid, [H, W, 1])
        rays = rays.reshape(H, W, 3).permute(2, 0, 1).unsqueeze(0).repeat(B, 1, 1, 1)
        print(rays.shape)
        # rays = rays.unsqueeze(0).repeat(B, 1, 1, 1)
        # print(grid.shape)
        print(cnn_layer(rays).shape)

        break
    break

In [ ]:

def _voxelize(space, voxel_size):
    X = torch.arange(space[0][0], space[0][1] + voxel_size[0]/8, voxel_size[0])
    Y = torch.arange(space[1][0], space[1][1] + voxel_size[1]/8, voxel_size[1])
    Z = torch.arange(space[2][0], space[2][1] + voxel_size[2]/8, voxel_size[2])
    # print(X.shape, Y.shape, Z.shape)

    grid_x, grid_y, grid_z = torch.meshgrid(X, Y, Z, indexing='ij')
    return torch.stack([grid_x.flatten(), grid_y.flatten(), grid_z.flatten()], dim=1)

In [ ]:
n_class, k = 2, 6
# layers = nn.ModuleList([
#     cvmpca.VoxelMHA(
#         embed_dim=embed_dim, num_heads=12, attention_dropout=0.1, dropout=0.1,
#         cameras=cameras
#     ) for _ in range(len(swin.embed_dims))
# ])
cls_embedding = nn.Embedding(n_class, embed_dim)

space = torch.tensor([[-11, 11], [0, 3], [-7, 7]])
voxel_size = [0.1, 0.2, 0.1]

def _original(indices, n_bins):
    X, Y, Z = n_bins

    x, y, z = (indices//Z)//Y, (indices//Z)%Y, indices%Z
    
    return torch.stack([x, y, z], dim=-1)

head = nn.Linear(embed_dim, n_class, bias=False)

next_voxel_size = np.array(voxel_size)/(3**0)

voxels = _voxelize(space, next_voxel_size)
voxels = voxels.unsqueeze(0).repeat(B, 1, 1)

camera_features = {}
for camera_key, feature_dict in visual_features.items():
    camera = cameras[camera_key]
    feature = list(feature_dict.values())[-1]

    *_, H, W = feature.shape
    pixel_coor = camera(voxels, [H, W, 1])  # (B, N, 2)

    bounding = torch.logical_and(
        torch.logical_and(pixel_coor[..., 0] >= 0, pixel_coor[..., 0] <= H),
        torch.logical_and(pixel_coor[..., 1] >= 0, pixel_coor[..., 1] <= W)
    )  # (B, N)

    # for grid_sample compatibility. (B, N, 1, 2)
    pixel_coor = pixel_coor.unsqueeze(-2)
    camera_feature = F.grid_sample(feature, pixel_coor, align_corners=True).squeeze(-1)
    camera_feature = camera_feature*bounding.unsqueeze(1)#.unsqueeze(0)  # (B, D, N)
    camera_feature = camera_feature.permute(0, 2, 1)  # (B, N, D)
    
    camera_features[camera_key] = camera_feature
    # camera_feature = camera_feature.reshape(B, C, K, N, -1)

for key, item in camera_features.items():
    print(key, item.shape)

voxel_features = torch.stack([item for item in camera_features.values()], dim=-1).sum(-1)
print(voxel_features.shape)

logits = head(voxel_features)
logits.shape

In [ ]:
_, indices = logits.view(B, -1).topk(k, dim=-1)
top_proposals_index = indices // n_class  # (B, K) which voxel
top_proposals_class = indices % n_class   # (B, K) which class
top_proposals_index.shape, top_proposals_class.shape

In [ ]:
selected_voxels = voxel_features.gather(1, top_proposals_index.unsqueeze(-1).repeat(1, 1, embed_dim))
selected_voxels.shape

In [ ]:
(voxel_features[torch.arange(1).unsqueeze(-1), top_proposals_index] == selected_voxels).all()

In [ ]:
net = cvmpca.CVMPCA(
    train_loader.dataset.captures,
    n_classes=len(train_loader.dataset.category_lookup)-1,  # ignore background
    space=[[-11, 11], [0, 3], [-7, 7]],
    voxel_size=[0.2, 0.3, 0.2],
    ratio=3,
)
# outputs = net(50, image_dicts)
print()

In [ ]:
image_dicts, object_list = batch
output_list, points = net(50, image_dicts)
# net.cuda()
# output_list, voxel_list = net(50, {key: item.cuda() for key, item in image_dicts.items()})
# B, L, C, P, _ = cls_list.shape

In [ ]:

indices = net._match(
    (output_list, points),
    object_list
)
print()

In [ ]:
net.training_step(batch, 0)

In [6]:
trainer = pl.Trainer(
    max_epochs=100, #precision='16-mixed',
    gradient_clip_val=35,
    log_every_n_steps=20,
    # accelerator="cpu"
    # profiler="simple",
    # detect_anomaly=True
)
net = cvmpca.CVMPCA(
    train_loader.dataset.captures,
    n_classes=len(train_loader.dataset.category_lookup)-1,
    space=[[-11, 11], [0, 3], [-7, 7]],
    voxel_size=[0.075, 0.2, 0.075],
    ratio=3,
)
# loss_func = SetCriterion(
#     num_ue=50, num_sbs=2, num_class=1,
#     num_layers=3, pc_range=[[-5, 5], [0, 3], [-5, 5]]
# )

trainer.fit(net, train_dataloaders=train_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                  | Params
--------------------------------------------------
0 | cameras | ModuleDict            | 0     
1 | swin    | Swin                  | 27.6 M
2 | fpn     | FeaturePyramidNetwork | 470 K 
3 | ray_pe  | Conv2d                | 384   
4 | blocks  | ModuleList            | 513 K 
5 | heads   | ModuleList            | 99.4 K
--------------------------------------------------
28.7 M    Trainable params
0         Non-trainable params
28.7 M    Total params
114.661   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
net.space[:, 0], net.space[:, 1]

In [ ]:
len(train_loader.dataset.category_lookup)

In [ ]:

net = cvmpca.CVMPCA(
    train_loader.dataset.captures,
    n_classes=len(train_loader.dataset.category_lookup),
    space=[[-11, 11], [0, 3], [-7, 7]],
    # voxel_size=[0.075, 0.2, 0.075],
    voxel_size=[0.2, .3, .2],
    ratio=3,
).cuda()


def _get_src_permutation_idx(indices):
    # permute predictions following indices
    batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
    src_idx = torch.cat([src for (src, _) in indices])
    return batch_idx, src_idx

for batch in train_loader:
    image_dicts, object_list = batch
    for obj in object_list:
        obj.category = obj.category.cuda()
        obj.position = obj.position.cuda()
        obj.los = obj.los.cuda()
    image_dicts = {
        key: item.cuda()
        for key, item in image_dicts.items()
    }
    break

optimizer = torch.optim.AdamW(net.parameters(), lr=5e-5, weight_decay=1e-4)
for i in range(1000):
    output_dict = net(200, image_dicts)
    # top_indices = output_dict['top_indices']

    index_list = net._match(output_dict['output_list'], object_list)
    # perform matching
    # with torch.no_grad():
    #     # B, K = top_indices.shape
    #     # print(output_dict['voxels'].shape)
    #     B, N = output_dict['output_list'][0]['position'].shape[:2]

    #     tgt_ids = torch.cat([tgt.category for tgt in object_list]).argmax(-1)
    #     tgt_pos = torch.cat([tgt.position for tgt in object_list])
    #     tgt_link = torch.cat([tgt.los for tgt in object_list]).sigmoid()

    #     # b_ids = torch.arange(B)
    #     out_prob = output_dict['output_list'][0]['classification'].flatten(0, 1).softmax(-1)
    #     cost_class = -out_prob[:, tgt_ids]
    #     # print(out_prob.shape, cost_class.shape)
        
    #     out_pos = output_dict['output_list'][0]['position'].flatten(0, 1)
    #     # print(out_pos, tgt_pos)
    #     cost_pos = torch.cdist(out_pos, tgt_pos, p=1)
    #     C = 0*cost_class + cost_pos
    #     # print(cost_class.shape, cost_pos.shape, C.shape)
    #     C = C.view(B, N, -1).cpu()

    #     sizes = [len(obj.category) for obj in object_list]

    #     # perform matching for each output-label pair in a batch
    #     indices = [linear_sum_assignment(c[i]) for i, c in enumerate(C.split(sizes, -1))]
    #     indices = [
    #         (torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64))
    #         for i, j in indices
    #     ]

    target_class = torch.cat([obj.category[i] for obj, (_, i) in zip(object_list, indices)], dim=0)#.argmax(-1)
    target_pos = torch.cat([obj.position[i] for obj, (_, i) in zip(object_list, indices)], dim=0)

    idx = _get_src_permutation_idx(index_list[0])
    # pred_class = output_dict['voxel_logits'][idx]
    # pred_pos = output_dict['voxels'][idx]
    pred_class = output_dict['output_list'][0]['classification'][idx]
    pred_pos = output_dict['output_list'][0]['position'][idx]
    
    target_classes = torch.full_like(output_dict['output_list'][0]['classification'], 0.)
    target_classes[idx] = target_class
    ce_loss = sigmoid_focal_loss(output_dict['output_list'][0]['classification'], target_classes, reduction='mean')

    pos_loss = F.l1_loss(pred_pos, target_pos, reduction='mean')
    loss = ce_loss + pos_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'{i} {loss.item():.4f} {pos_loss.item():.4f}')
    # break


In [ ]:
output_dict['output_list'][0]['position'].shape

In [ ]:
output_dict = net(200, image_dict)

In [ ]:
index_list = net._match(output_dict['output_list'], object_list)

In [ ]:
net.training_step(batch, 0)